In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.8 MB/s 
     |████████████████████████████████| 163 kB 85.3 MB/s 
     |████████████████████████████████| 7.6 MB 75.1 MB/s 


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
from transformers import ElectraModel, ElectraTokenizer

In [ ]:
# model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

In [ ]:
from transformers import ElectraForSequenceClassification

In [ ]:
# efsc = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=128)

Downloading:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [ ]:
# efsc.parameters

In [ ]:
import numpy as np
import pandas as pd
device = torch.device('cuda')

In [ ]:
train_data = pd.read_csv('/content/gdrive/MyDrive/da/open/train.csv')

In [ ]:
train_txt = train_data['overview']
train_cat3 = train_data['cat3']

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(train_cat3)
train_target = label_encoder.transform(train_cat3)

In [ ]:
# model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)

In [ ]:
class SampleData(Dataset):
    def __init__(self, X, target, tokenizer):
        super(SampleData, self).__init__()
        self.X = X.to_numpy()
        
        self.target = target

    def __len__(self):
        return len(self.target)

    def __getitem__(self, item):
        encoding = tokenizer(self.X[item],
                                  return_token_type_ids = True,
                                  return_attention_mask = True,
                                  truncation = True,
                                  padding='max_length',
                                  max_length = 300,
                                  return_tensors = 'pt')
        return encoding['input_ids'][0], encoding['attention_mask'][0], self.target[item]

In [ ]:
# train_X = train_txt.iloc[:20]
# train_y = train_target[:20]

In [ ]:
# td = SampleData(train_X, train_y, tokenizer)

In [ ]:
# tl = DataLoader(td, batch_size=5, shuffle=False)

In [ ]:
# a,b,c = next(iter(tl))

In [ ]:
# a

In [ ]:
train_data = SampleData(train_txt, train_target, tokenizer)

In [ ]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

In [ ]:
label_encoder.classes_

array(['5일장', 'ATV', 'MTB', '강', '게스트하우스', '계곡', '고궁', '고택', '골프', '공연장',
       '공예,공방', '공원', '관광단지', '국립공원', '군립공원', '기념관', '기념탑/기념비/전망대',
       '기암괴석', '기타', '기타행사', '농.산.어촌 체험', '다리/대교', '대중콘서트', '대형서점',
       '도립공원', '도서관', '동굴', '동상', '등대', '래프팅', '면세점', '모텔', '문', '문화관광축제',
       '문화원', '문화전수시설', '뮤지컬', '미술관/화랑', '민물낚시', '민박', '민속마을', '바/까페',
       '바다낚시', '박람회', '박물관', '발전소', '백화점', '번지점프', '복합 레포츠', '분수', '빙벽등반',
       '사격장', '사찰', '산', '상설시장', '생가', '서비스드레지던스', '서양식', '섬', '성',
       '수련시설', '수목원', '수상레포츠', '수영', '스노쿨링/스킨스쿠버다이빙', '스카이다이빙', '스케이트',
       '스키(보드) 렌탈샵', '스키/스노보드', '승마', '식음료', '썰매장', '안보관광', '야영장,오토캠핑장',
       '약수터', '연극', '영화관', '온천/욕장/스파', '외국문화원', '요트', '윈드서핑/제트스키',
       '유람선/잠수함관광', '유명건물', '유스호스텔', '유원지', '유적지/사적지', '이색거리', '이색찜질방',
       '이색체험', '인라인(실내 인라인 포함)', '일반축제', '일식', '자동차경주', '자연생태관광지',
       '자연휴양림', '자전거하이킹', '전문상가', '전시관', '전통공연', '종교성지', '중식', '채식전문점',
       '카약/카누', '카지노', '카트', '컨벤션', '컨벤션센터', '콘도미니엄', '클래식음악회', '클럽',
       '

In [ ]:
class ClassifierModel(nn.Module):
    def __init__(self, electra):
        super(ClassifierModel, self).__init__()
        self.model = electra
        self.dense = nn.Linear(768*250, 256)
        self.output = nn.Linear(256, 128)

    def forward(self, x, mask):
        x = self.model(x , mask)
        x = x[0].view(-1,153600)
        x = self.dense(x)
        x = nn.functional.relu(x)
        x = self.output(x)
        x = nn.functional.softmax(x)
        return x

In [ ]:
model_c = ElectraForSequenceClassification.from_pretrained('monologg/koelectra-base-v3-discriminator', num_labels=128).to(device)

Downloading:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [ ]:
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# c_model = ClassifierModel(model).to(device)
optimizer = torch.optim.Adam(model_c.parameters(), lr=5e-6)

# batch_size = 32
# scheduler = get_cosine_schedule_with_warmup(
#             optimizer,
#             num_warmup_steps=int(len(train_loader) * 20 * 0.1),
#             num_training_steps= len(train_loader) * 20)
epochs = 25


In [ ]:
len(train_loader)

531

In [ ]:
from tqdm import tqdm

In [ ]:
losses = []
accuracies = []

for i in range(epochs):
  print('\n'+'===='*5+str(i+1)+'epoch '+'===='*5+'\n')
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0


  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model_c(input_ids_batch.to(device), attention_masks_batch.to(device))[0]

        
    loss = nn.functional.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()
    # scheduler.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("\nBatch Loss:", total_loss, "Accuracy:", correct.float() / total, '\n')
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)


====================1epoch ====================



 38%|███▊      | 100/266 [01:25<02:21,  1.17it/s]


Batch Loss: 345.5659122467041 Accuracy: tensor(0.3259, device='cuda:0') 



 75%|███████▌  | 200/266 [02:51<00:56,  1.16it/s]


Batch Loss: 663.1267957687378 Accuracy: tensor(0.3447, device='cuda:0') 



...


Batch Loss: 16.345274783670902 Accuracy: tensor(0.9723, device='cuda:0') 



 75%|███████▌  | 200/266 [02:51<00:56,  1.17it/s]


Batch Loss: 30.170458123087883 Accuracy: tensor(0.9753, device='cuda:0') 



100%|██████████| 266/266 [03:48<00:00,  1.17it/s]

Train Loss: 39.218274258077145 Accuracy: tensor(0.9760, device='cuda:0')


In [ ]:
test_data = pd.read_csv('/content/gdrive/MyDrive/da/open/test.csv')

In [ ]:
test_dt = SampleData(test_data['overview'], test_data['overview'], tokenizer)

In [ ]:
test_ld = DataLoader(test_dt, shuffle=False)

In [ ]:
with torch.no_grad():
    result = []    
    
    for a,b,c in tqdm(test_ld):
        model_c.eval()

        re = model_c(a.to(device), b.to(device))
        result.append(torch.max(re[0], axis=1)[1])

100%|██████████| 7280/7280 [02:20<00:00, 51.75it/s]


In [ ]:
result_ = list(map(lambda x : label_encoder.classes_[x.item()], result))

In [ ]:
submit_df = pd.read_csv('/content/gdrive/MyDrive/da/open/sample_submission.csv')
submit_df['cat3'] = result_
submit_df.to_csv('/content/gdrive/MyDrive/da/open/submit_electra_20ep.csv', index=False)

In [ ]:
torch.save(model_c.state_dict, '/content/gdrive/MyDrive/da/open/electra_20ep.pt')